In [1]:
import time
import csv
import pandas as pd
import math as m

In [2]:
data = pd.read_csv('cleaned_data_collision_division.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)

## Data and values

In [3]:
# icao values
icao_values = list(set(data.icao))

# uav loc df
uav_cols = ['ui', 'timestamp', 'longitude', 'latitude', 'altitude', 'heading']
uav_loc_df = pd.DataFrame(columns=uav_cols)

## Algorithm implementation

### Functions

In [145]:
def header_warning(uav, plane, uav_turn_angle):
    # uav and plane passed in as array with lat, long, alt, header, km_per_long
    # Finding Quadrants
    x = plane[0] - uav[0]
    y = plane[1] - uav[1]

    # reference header angle for turning
    ref_header = plane[3] - uav[3]

    if (x < 0) & (y > 0): # top left quadrant
        if (ref_header >= 180) & (ref_header <= 360):
            return(uav[3])
        else:
            uav[3] += uav_turn_angle
            uav[3] = uav[3]%360
            return(uav[3])
    elif (x > 0) & (y > 0): # top right quadrant
        if (ref_header >= 0) & (ref_header <= 180):
            return(uav[3])
        else:
            uav[3] -= uav_turn_angle
            uav[3] = uav[3]%360
            return(uav[3])
    elif (x < 0) & (y < 0): # bottom left quadrant
        if (ref_header >= 90) & (ref_header <= 360):
            return(uav[3])
        else:
            uav[3] -= uav_turn_angle
            uav[3] = uav[3]%360
            return(uav[3])
    elif (x > 0) & (y < 0): # bottom right quadrant
        if (ref_header >= 0) & (ref_header <= 270):
            return(uav[3])
        else:
            uav[3] -= uav_turn_angle
            uav[3] = uav[3]%360
            return(uav[3])


In [25]:
def uav_loc_to_df(uav, timestamp, message):
    messages.append(message)
    uav_dict[timestamp] = uav
    

In [126]:
def altitude_warning(uav_altitude, plane_altitude, descent_speed):
    if uav_altitude - plane_altitude < 0:
        uav_altitude -= descent_speed
    else:
        uav_altitude += descent_speed
    return(uav_altitude)

In [6]:
def convert_to_ellip(input, km_per_lat):
    ellip = {}
    for key in input:
        ## Update cart Dict with Cartesian values
        ellip.update({key:[input[key][0]/input[key][4], input[key][1]/km_per_lat, input[key][2], input[key][3]]})
    return(ellip)


In [7]:
def convert_to_cart(input, km_per_lat):
    cart = {}
    for key in input:
        ## Longitude speed calculation
        phi = input[key][0] * (m.pi/180)
        km_per_long = km_per_lat * m.cos(phi)

        ## Update cart Dict with Cartesian values
        cart.update({key:[input[key][0]*km_per_long, input[key][1]*km_per_lat, input[key][2], input[key][3], km_per_long]})
    return(cart)

In [8]:
def plane_locator(uav_starttime):
    """ Params: uav_starttime - When the UAV starts flying
        Returns: all active planes at that time stamp
    """
    plane_locs = {}
    for icao in icao_values:
        key = icao
        try:
            lat,long,alt,head = data[['latitude', 
                                      'longitude', 
                                      'altitude_(m)', 
                                      'heading']].loc[(data.timestamp==uav_starttime) 
                                                      & (data.icao==icao)].values[0]
        except:
            lat = 0
            long = 0
            alt = 0
            head = 0
            key = None
        plane_locs[key] = [lat, long, alt, head]
    active_plane_locs = {k: v for k, v in plane_locs.items() if k is not None}
    return active_plane_locs

In [9]:
def move_distance(heading, speed):
    x = speed * m.cos(heading)
    y = speed * m.sin(heading)
    return(x, y)

In [115]:
# def find_distance(uav, plane, uav_speed, uav_descent_speed, uav_turn_angle):
#     i = 1
#     uav_messages = []
#     for uav_key in uav:
#             for craft_key in plane:
#                 dist_xy = m.sqrt((plane[craft_key][0] - uav[uav_key][0])**2 + (plane[craft_key][1] - uav[uav_key][1])**2)
#                 dist_alt = abs(plane[craft_key][2] - uav[uav_key][2])

#             if dist_xy <= 10:
#                 if dist_alt <= 500:
#                     if dist_xy <= 5:
#                         message = "Critical! You are " + {round(dist_xy, 3)} + "km away!"
#                         uav_messages.append(message)
#                     else:
#                         message = "Warning! You are " + {round(dist_xy, 3)} + "km away!"
#                         uav_messages.append(message)
#                     #header_warning(uav[uav_key], plane[craft_key], uav_turn_angle)
#                     uav[uav_key][2] = altitude_warning(uav[uav_key][2], plane[craft_key][2], uav_descent_speed)
#                 elif dist_alt > 500:
#                     x, y = move_distance(uav[uav_key][3], uav_speed)
#                     uav[uav_key][0] += (x/1222)
#                     uav[uav_key][1] += (y/1222)
#             else:
#                 message = "Clear"
#                 x, y = move_distance(uav[uav_key][3], uav_speed)
#                 uav[uav_key][0] += (x/1222)
#                 uav[uav_key][1] += (y/1222)
#                 uav_messages.append(message)
#     print(uav_messages)
#     return(uav, message)

In [175]:
def find_distance(uav, plane, uav_speed, uav_descent_speed, uav_turn_angle):
    uav_messages = []
    for uav_key in uav:
        for craft_key in plane:
            dist_xy = m.sqrt((plane[craft_key][0] - uav[uav_key][0])**2 + (plane[craft_key][1] - uav[uav_key][1])**2)
            dist_alt = abs(plane[craft_key][2] - uav[uav_key][2])

            if dist_xy <= 10:
                if dist_alt <= 500:
                    if dist_xy <= 5:
                        message = f"Critical! You are {round(dist_xy, 3)} km away!"
                        print(message)
                        uav_messages.append(message)
                    else:
                        message = f"Warning! You are {round(dist_xy, 3)} km away!"
                        print(message)
                        uav_messages.append(message)
                    uav[uav_key][3] = header_warning(uav[uav_key], plane[craft_key], uav_turn_angle)
                    x, y = move_distance(uav[uav_key][3], uav_speed)
                    uav[uav_key][0] += (x/1222)
                    uav[uav_key][1] += (y/1222)
                    uav[uav_key][2] = altitude_warning(uav[uav_key][2], plane[craft_key][2], uav_descent_speed)
                elif dist_alt > 500:
                    x, y = move_distance(uav[uav_key][3], uav_speed)
                    uav[uav_key][0] += (x/1222)
                    uav[uav_key][1] += (y/1222)
            else:
                message = "Clear"
                x, y = move_distance(uav[uav_key][3], uav_speed)
                uav[uav_key][0] += (x/1222)
                uav[uav_key][1] += (y/1222)
                uav_messages.append(message)
    
    for msg in uav_messages:
        if 'Critical!' in msg:
            message = msg
        elif 'Warning!' in msg:
            message = msg

    return(uav, message)

# Main Loop

In [180]:
%%time

# Globals
km_per_lat = 111.2403
warning = False
alt_warning = False
uav_speed = 25
uav_descent_speed = 0.002
uav_turn_angle = 10

# uav start position
uav = {'uav_icao_1': [38.879638, 282.379883, 10965.180000, 218.927544]}

# Change this based on when the UAV starts flying
uav_starttime = 1541709072

# how long we want the drone to fly in seconds
flight_time = 240

# Dict that holds all of the UAV values
uav_dict = {}
messages = []
uav_messages_dict = {}

# The main loop (change the while condition based on if UAV has arrived at end location)
i = 0
while i < flight_time:
    timestamp = uav_starttime+i
    print("Timestamp: ", timestamp)
    
    # Required functions
    active_planes = plane_locator(uav_starttime)
    cart = convert_to_cart(active_planes, km_per_lat)
    uav = convert_to_cart(uav, km_per_lat)
    uav, message = find_distance(uav, cart, uav_speed, uav_descent_speed, uav_turn_angle)
    uav = convert_to_ellip(uav, km_per_lat)
    uav_loc_to_df(uav, timestamp, message)
    
    i+=1

Timestamp:  1541709072
Timestamp:  1541709073
Timestamp:  1541709074
Timestamp:  1541709075
Timestamp:  1541709076
Timestamp:  1541709077
Timestamp:  1541709078
Timestamp:  1541709079
Timestamp:  1541709080
Timestamp:  1541709081
Timestamp:  1541709082
Timestamp:  1541709083
Timestamp:  1541709084
Timestamp:  1541709085
Timestamp:  1541709086
Timestamp:  1541709087
Timestamp:  1541709088
Timestamp:  1541709089
Timestamp:  1541709090
Timestamp:  1541709091
Timestamp:  1541709092
Timestamp:  1541709093
Timestamp:  1541709094
Timestamp:  1541709095
Timestamp:  1541709096
Timestamp:  1541709097
Timestamp:  1541709098
Timestamp:  1541709099
Timestamp:  1541709100
Timestamp:  1541709101
Timestamp:  1541709102
Timestamp:  1541709103
Timestamp:  1541709104
Timestamp:  1541709105
Timestamp:  1541709106
Timestamp:  1541709107
Timestamp:  1541709108
Timestamp:  1541709109
Timestamp:  1541709110
Timestamp:  1541709111
Timestamp:  1541709112
Timestamp:  1541709113
Timestamp:  1541709114
Timestamp: 

## Outputting the data

In [181]:
# Converts the uav dicts to a dataframe
timestamps = []
uav_icao = []
latitudes = []
longitudes = []
altitudes = []
headings = []

for k,v in uav_dict.items():
    timestamps.append(k)
    for k1, v1 in v.items():
        uav_icao.append(k1)
        latitudes.append(v1[0])
        longitudes.append(v1[1])
        altitudes.append(v1[2])
        headings.append(v1[3])

d = {'uav_icao': uav_icao, 'timestamp': timestamps,  
    'latitude': latitudes, 'longitudes': longitudes, 
    'altitude': altitudes, 'heading': headings, 
    'message': messages}

uav_df = pd.DataFrame.from_dict(d)

columns = ['uav_icao', 'timestamp', 'latitude', 'longitudes', 'altitude', 'heading', 'message']

uav_df = uav_df.reindex(columns=columns)

In [182]:
# Outputs a csv of the uav data
uav_df.to_csv('uav_data.csv')

In [183]:
uav_df

,uav_icao,timestamp,latitude,longitudes,altitude,heading,message
0,uav_icao_1,1541709072,38.882254,282.376820,10965.18,218.927544,Clear
1,uav_icao_1,1541709073,38.884871,282.373757,10965.18,218.927544,Clear
2,uav_icao_1,1541709074,38.887488,282.370695,10965.18,218.927544,Clear
3,uav_icao_1,1541709075,38.890104,282.367632,10965.18,218.927544,Clear
4,uav_icao_1,1541709076,38.892721,282.364569,10965.18,218.927544,Clear
5,uav_icao_1,1541709077,38.895338,282.361506,10965.18,218.927544,Clear
6,uav_icao_1,1541709078,38.897955,282.358443,10965.18,218.927544,Clear
7,uav_icao_1,1541709079,38.900572,282.355381,10965.18,218.927544,Clear
8,uav_icao_1,1541709080,38.903189,282.352318,10965.18,218.927544,Clear
9,uav_icao_1,1541709081,38.905806,282.349255,10965.18,218.927544,Clear
